# StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

---
## Load Libraries and Python Options:
### Load Libraries:

In [ ]:
import sys
sys.path.append('../Core')
from StreamStats_API_Scraper import*

### Specify the state abbreviation and location of the json file: 

In [ ]:
state = 'NY'  # The state abbreviation in uppercase

root_dir = pl.Path(r'C:\Users\sputnam\Desktop\RI_FEMA_II\River')
inputs_dir = root_dir/'Inputs'
outputs_dir = root_dir/'Outputs'

riv_gauges = 'riv_gauges_parsed.json'  # Specify the location of the shapefile/json containing the lat/lon of points on the stream grid

## Options:
use_epsg = '4326'    # Specify a consistent coordinate reference system
get_flow = False     # If state is WI set to False
print_status = True  # Option to show print statements

### Load the json:

In [ ]:
with open(inputs_dir/riv_gauges) as f:
    dic = json.load(f)
    
print('{0} gages'.format(len(dic['features'])))    
dic['features'][0]

#### Select for state of interest:

In [ ]:
select = {}
for feature in dic['features']:
    if feature['properties']['State'] == state:
        select[feature['properties']['Gauge_No']] = feature['geometry']['coordinates']
print('{0} gages for {1}'.format(len(list(select.keys())), state))  

##### Save the state of interest to a geojson file (optional):

In [ ]:
coord_df = pd.DataFrame(data={'ID_Num': [k for k in select.keys()], 'Lon':[v[0] for v in select.values()],'Lat':[v[1] for v in select.values()]})
coord_df['geometry'] = list(zip(coord_df['Lon'], coord_df['Lat']))
coord_df['geometry'] = coord_df['geometry'].apply(Point)
gdf = gpd.GeoDataFrame(coord_df, geometry='geometry', crs={'init': 'epsg:%s' %use_epsg})
with open(outputs_dir/'{0}.geojson'.format(state), 'w') as f:
    geojson.dump(gdf, f)  

---
## Run the API tool for each point:

In [ ]:
polyg = {}   # Dictionary to store the catchment polygons (catchment boundaries) 
ffdata = {}  # Dictionary to store the outlet flow frequency data dictionaries
errors = []

for ID_Num, geom in select.items():
    try:
        lon, lat = geom[0], geom[1]  #  Longitude and latitude for each shapely point and the confluence number
        if print_status: 
            print("Lat/Lon/Confluence:", lat, lon, ID_Num)
        polyg[ID_Num], ff_json  = SS_scrape(state, lon, lat, use_epsg, print_status) 
        if get_flow: 
            ffdata[ID_Num] = get_peaks(ff_json) 
            polyg[ID_Num]['features'][0]['ffcurve']  = ffdata[ID_Num]
        with open(os.path.join(allresults,'StreamStats_Polygons_{0}.geojson'.format(str(ID_Num))), 'w') as f:
            dump(polyg[ID_Num], f)
    except:
        print('could not process data {}'.format(ID_Num))
        errors.append(ID_Num) 

---
## Load the results (NOT CHECKED 12-2-19):

In [ ]:
files = load_files(allresults)
gdf2, ffdic=load_results(files, use_epsg)

In [ ]:
gdf.drop(gdf.loc[gdf['Id'][gdf2['ID_Num']]].index, inplace=True)
gdf.index.values

###  Save:

##### The flow frequency data as a CSV:

In [ ]:
#Run this function to construct the summary table for all outlet locations
if get_flow: ff_df=ff_summary(ffdic) 
    
#Save the results as a csv
if get_flow: ff_df.to_csv(os.path.join(path,'StreamStats_FlowFrequency.csv')) 

##### The catchment polygons as a Shapefile:

In [ ]:
#Export the geodataframe as a shapefile
gdf2 = convert_attr(gdf2)
gdf2.to_file(filename = os.path.join(path,'StreamStats_Polygons.shp')) 

##### The catchment polygons as a geojson:

In [ ]:
with open(os.path.join(path,'StreamStats_Polygons.geojson'), 'w') as f:
     dump(gdf2, f)  

# END